#Chatbot using Seq2Seq LSTM models

code reference - 
https://medium.com/predict/creating-a-chatbot-from-scratch-using-keras-and-tensorflow-59e8fc76be79 

In this notebook, we will assemble a seq2seq LSTM model using Keras Functional API to create a working Chatbot which would answer questions asked to it.

Chatbots have become applications themselves. You can choose the field or stream and gather data regarding various questions. We can build a chatbot for an e-commerce webiste or a school website where parents could get information about the school.

Messaging platforms like Allo have implemented chatbot services to engage users. The famous Google Assistant, Siri, Cortana and Alexa may have been build using simialr models.

So, let's start building our Chatbot.

## 1) Importing the packages
We will import TensorFlow and our beloved Keras. Also, we import other modules which help in defining model layers.

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
print('done')

done


#Preprocessing the data
A) Download the data
The dataset hails from chatterbot/english on Kaggle.com by kausr25.

https://www.kaggle.com/kausr25/chatterbotenglish 

 It contains pairs of questions and answers based on a number of subjects like food, history, AI etc.

The raw data could be found from this repo -> https://github.com/shubham0204/Dataset_Archives

In [ ]:
!wget https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true -O chatbot_nlp.zip
!unzip chatbot_nlp.zip

--2021-08-15 23:24:06--  https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip [following]
--2021-08-15 23:24:06--  https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip [following]
--2021-08-15 23:24:06--  https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

##B) Reading the data from the files
We parse each of the .yaml files.

Concatenate two or more sentences if the answer has two or more of them.
Remove unwanted data types which are produced while parsing the data.
Append <START> and <END> to all the answers.
Create a Tokenizer and load the whole vocabulary ( questions + answers ) into it.

In [ ]:
from tensorflow.keras import preprocessing , utils
import os
import yaml

dir_path = 'chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])


answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )


answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )


tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1894


#C) Preparing data for Seq2Seq model
Our model requires three arrays namely encoder_input_data, decoder_input_data and decoder_output_data.

For encoder_input_data :

Tokenize the questions. Pad them to their maximum length.
For decoder_input_data :

Tokenize the answers. Pad them to their maximum length.
For decoder_output_data :

Tokenize the answers. Remove the first element from all the tokenized_answers. This is the <START> element which we added earlier.
[ ]


In [ ]:

from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary

#p = tokenize( questions + answers )
#model = Word2Vec( p[ 0 ] ) 

#embedding_matrix = np.zeros( ( VOCAB_SIZE , 100 ) )
#for i in range( len( tokenizer.word_index ) ):
    #embedding_matrix[ i ] = model[ vocab[i] ]

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(564, 22) 22
(564, 74) 74
(564, 74, 1894)


#3) Defining the Encoder-Decoder model
The model will have Embedding, LSTM and Dense layers. The basic configuration is as follows.

* 2 Input Layers : One for encoder_input_data and another for 
decoder_input_data.
* Embedding layer : For converting token vectors to fix sized dense vectors. ( Note : Don't forget the mask_zero=True argument here )
* LSTM layer : Provide access to Long-Short Term cells.
Working :

  -

1. The encoder_input_data comes in the Embedding layer (encoder_embedding).
2. The output of the Embedding layer goes to the LSTM cell which produces 2 state vectors ( h and c which are encoder_states )
3. These states are set in the LSTM cell of the decoder.
4. The decoder_input_data comes in through the Embedding layer.
5. The Embeddings goes in LSTM cell ( which had the states ) to produce seqeunces.

<center><img style="float: center;" src="https://cdn-images-1.medium.com/max/1600/1*bnRvZDDapHF8Gk8soACtCQ.gif"></center>


Image credits to [Hackernoon](https://hackernoon.com/tutorial-3-what-is-seq2seq-for-text-summarization-and-why-68ebaa644db0).

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 22)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 74)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 22, 200)      378800      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 74, 200)      378800      input_2[0][0]                    
______________________________________________________________________________________________

#4) Training the model
We train the model for a number of epochs with RMSprop optimizer and categorical_crossentropy loss function.

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 
model.save( 'model.h5' ) 


Epoch 1/150
12/12 [==============================] - 13s 42ms/step - loss: 1.2991
Epoch 2/150
12/12 [==============================] - 0s 40ms/step - loss: 1.1168
Epoch 3/150
12/12 [==============================] - 0s 40ms/step - loss: 1.0956
Epoch 4/150
12/12 [==============================] - 0s 39ms/step - loss: 1.0767
Epoch 5/150
12/12 [==============================] - 0s 39ms/step - loss: 1.0571
Epoch 6/150
12/12 [==============================] - 0s 40ms/step - loss: 1.0399
Epoch 7/150
12/12 [==============================] - 0s 39ms/step - loss: 1.0246
Epoch 8/150
12/12 [==============================] - 0s 40ms/step - loss: 1.0118
Epoch 9/150
12/12 [==============================] - 0s 40ms/step - loss: 0.9990
Epoch 10/150
12/12 [==============================] - 0s 39ms/step - loss: 0.9857
Epoch 11/150
12/12 [==============================] - 0s 40ms/step - loss: 0.9717
Epoch 12/150
12/12 [==============================] - 0s 39ms/step - loss: 0.9573
Epoch 13/150
12/12 [====

#5) Defining inference models
We create inference models which help in predicting answers.

Encoder inference model : Takes the question as input and outputs LSTM states ( h and c ).

Decoder inference model : Takes in 2 inputs, one are the LSTM states ( Output of encoder model ), second are the answer input seqeunces ( ones not having the <start> tag ). It will output the answers for the question which we fed to the encoder model and its state values.

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

#6) Talking with our Chatbot
First, we define a method str_to_tokens which converts str questions to Integer tokens with padding.

In [ ]:
import re
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        try:
          word = re.sub('\W+','', word )
          tokens_list.append( tokenizer.word_index[ word ] ) 
        except:
          continue
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


First, we take a question as input and predict the state values using enc_model.

We set the state values in the decoder's LSTM.

Then, we generate a sequence which contains the <start> element.

We input this sequence in the dec_model.

We replace the <start> element with the element which was predicted by the dec_model and update the state values.

We carry out the above steps iteratively till we hit the <end> tag or the maximum answer length.

In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : what are you?
 i am just an artificial intelligence end
Enter question : thats cool your cool.
 i am not really into football end
Enter question : lol
 robots rule end


KeyboardInterrupt: ignored